# Test the initial installation of a Rhino client

#### Prerequisites
1. Have an active SSO login
2. Have an active Rhino Cloud user with the appropriate access
3. Have the path to the external data store (e.g. S3, GCS)
4. Copy the accompanying dataset.csv file over to the external data store

### 1. Initialization and Login

In [2]:
from getpass import getpass

import rhino_health as rh
from rhino_health.lib.endpoints.code_object import CodeObjectCreateInput


### 2. Initialize the input values

In [19]:
USERNAME = "dan@rhinohealth.com"
# DATASTORE_PATH = "/rhino_data/external/s3/hc/encounters.csv"
DATASTORE_PATH = "/rhino_data/validation/dataset.csv"
CURRENT_WORKING_DIR = "/Users/danielrunt/PycharmProjects/user-resources/sandbox/site-testing"
DEV_URL = "https://dev.rhinohealth.com/api"

### 3 Log in

In [11]:
print("Logging In")
session = rh.login(username=USERNAME, password=getpass(), rhino_api_url=DEV_URL)
#session = rh.login(authentication_details={"sso_access_token": TOKEN, "sso_provider": "google"}, rhino_api_url=DEV3_URL)
print("Logged In")

Logging In
Logged In


### 4 Create a project

In [12]:
from rhino_health.lib.endpoints.project.project_dataclass import ProjectCreateInput
user = session.current_user

project = session.project.get_project_by_name("Validation Project")
if project is None:
    new_project = ProjectCreateInput(
        name="Validation Project",
        description="Project for Site Installation Validation",
        type="Validation",
        primary_workgroup_uid=user.primary_workgroup_uid
    )
    project = session.project.add_project(new_project)

### 5 Create a dataset and schema using external datasource access

In [15]:
from rhino_health.lib.endpoints.dataset.dataset_dataclass import DatasetCreateInput

dataset = session.dataset.get_dataset_by_name("Validation Dataset")
if dataset is None:
    dataset_input = DatasetCreateInput(
                name="Validation Dataset",
                description="",
                project_uid=project.uid,
                workgroup_uid=project.primary_workgroup_uid,
                data_schema_uid = None,
                csv_filesystem_location=DATASTORE_PATH,
                method="filesystem",
                is_data_deidentified=True,
            )
    dataset = session.dataset.add_dataset(dataset_input)

edit the schema

In [42]:
def convert_schema_fields_to_csv_list(schema_fields):
    field_names = schema_fields.root[0].model_dump().keys()
    columns = []

    for schema_field in schema_fields.root:
        columns.append([schema_field.model_dump()[field] for field in field_names])

    transposed_rows = zip(*columns)
    result = []

    for row in transposed_rows:
        result.append(','.join(map(str, row)))

    return result

In [45]:
# from rhino_health.lib.endpoints.data_schema.data_schema_dataclass import SchemaField,SchemaFields,DataSchemaCreateInput
# import csv
# import io
#
# bmi_schema = session.data_schema.get_data_schema_by_name("Validation Dataset schema")
#
# schema_fields = bmi_schema.schema_fields
# bmi_field = SchemaField(
#     name="BMI",
#     description='',
#     identifier='',
#     role="Input",
#     type="Float",
#     type_params=None,
#     units='',
#     may_contain_phi=False,
#     permissions='default'
# )
# schema_fields.root.append(bmi_field)
# csv_list = convert_schema_fields_to_csv_list(schema_fields)
#
# data_schema_input = DataSchemaCreateInput(
#     name="Validation Dataset schema",
#     description="",
#     primary_workgroup_uid=project.primary_workgroup_uid,
#     project=project.uid,
#     base_version_uid=bmi_schema.uid,
#     add_version_if_exists=True,
#     schema_fields=csv_list
# )
# print(data_schema_input)
# bmi_schema = session.data_schema.create_data_schema(data_schema_input,return_existing=False,add_version_if_exists=True)

DataSchemaCreateInput session=None name='Validation Dataset schema' description='' base_version_uid='23523498-4ffe-4251-bd1e-648bfb8c023a' version=1 schema_fields=['SeriesUID,Gender,Height,Weight,Pneumonia,__Notes__,BMI', ',,,,,,', ',,,,,System generated column for user notes,', 'Input,Input,Input,Input,Input,Input,Input', 'String,String,Float,Float,Boolean,String,Float', 'None,None,None,None,None,None,None', ',,,,,,', 'False,False,False,False,False,False,False', 'default,default,default,default,default,default,default'] file_path=None primary_workgroup_uid='97d7456b-baef-4d88-bfb1-afd4975eaff2' project_uid='9c83187f-a6a8-4c53-8ed0-6088ea24adb2' project_uids=None
Exception: Failed to make request
Status is 400, Error: , Content is b'{"status":"error","payload_type":"validation_error","message":["Error performing schema validation: Error importing Data Schema: invalid row label in row 0: SeriesUID"],"error":["Error performing schema validation: Error importing Data Schema: invalid row l

### 6 Run Generalized Compute Code

In [16]:
bmi_calculation = "df['BMI'] = df['Weight'] / df['Height']**2"
output_schema_uid = session.data_schema.get_data_schema_by_name("Validation Dataset schema").uid

output_dataset, run_results = dataset.run_code(bmi_calculation, output_data_schema_uid = output_schema_uid, output_dataset_names_suffix = " with BMI")
run_results.dict()

Waiting for code run to complete (0 hours 0 minutes and a second)
Done.


{'creator_uid': '771a2617-8134-4a56-854a-b94407bb561a',
 'created_at': 'Dec 26, 2024 09:06:05PM',
 'uid': '51bb810f-cf04-4aa2-b4fd-2df4173689d2',
 'action_type': 'Run',
 'status': <CodeRunStatus.COMPLETED: 'Completed'>,
 'start_time': 'Dec 26, 2024 09:06:05PM',
 'end_time': 'Dec 26, 2024 09:06:12PM',
 'input_dataset_uids': [[['b8a2d6e5-cb70-42c0-a3b4-72251c8280d2']]],
 'output_dataset_uids': [[['3d30708e-3622-42d6-aecf-95c39641b8d8']]],
 'code_object_uid': '81ff709b-73b1-4d3c-bcaa-d0f62901ce1a',
 'results_info': {'tasks_errors_and_warnings': [{'errors': [],
    'warnings': []}],
  'num_dataset_import_failures': [0]},
 'results_report': None,
 'report_images': [],
 'paths_to_model_params': None}

### 7 Use NVFlare to Infer a Model With a Code Object

In [24]:
from rhino_health.lib.endpoints.code_object.code_object_dataclass import (
    CodeObject,
    CodeObjectRunInput,
    CodeTypes,
    CodeLocation,
    CodeExecutionMode,
    RequirementMode,
)
import os

data_schema = session.data_schema.get_data_schema_by_name("Validation Dataset schema")
code_object = session.code_object.get_code_object_by_name("NVFlare Validation Code")
if code_object is None:
    requirements = [
        "nvflare==2.0.5",
        "pandas==1.4.1",
        "torch>=1.10,<1.11",
        "torchvision>=0.11.1,<0.12",
        "protobuf<3.21"
    ]
    zip_path = "'/Users/danielrunt/PycharmProjects/user-resources/tutorials/tutorial_1/prediction-model"
    code_object_input = CodeObjectCreateInput(
        name="NVFlare Validation Code",
        description="",
        input_data_schema_uids=[data_schema.uid],
        output_data_schema_uids=[],
        project_uid=project.uid,
        code_type=CodeTypes.NVIDIA_FLARE_V2_0,
        config={
            "code_execution_mode": CodeExecutionMode.AUTO_CONTAINER_NVFLARE,
            "python_version": "3.8",
            "requirements": requirements,
            "requirements_mode": RequirementMode.PYTHON_PIP,
            "folder_path": zip_path,
            "code_location": CodeLocation.S3_MULTIPART_ZIP
        }
    )
    code_object = session.code_object.create_code_object(
        code_object_input, return_existing=False, add_version_if_exists=True
    )
    code_object.wait_for_build(1200)
    print(f"Finished Uploading New Code")

input_dataset_uids = [output_dataset[0].uid]
code_run_input = CodeObjectRunInput(
    code_object_uid = code_object.uid,
    input_dataset_uids = [input_dataset_uids],
    output_dataset_naming_templates = ["_infer"],
    timeout_seconds = 1200
)

session.code_object.run_code_object(code_run_input)

Waiting for code object build to complete (0 hours 0 minutes and a second)
Waiting for code object build to complete (0 hours 0 minutes and 31 seconds)
Done.
Finished Uploading New Code
Exception: Failed to make request
Status is 400, Error: , Content is b'{"status":"error","payload_type":"validation_error","message":["Cannot perform this action on an CodeObject that is not built yet"],"error":["Cannot perform this action on an CodeObject that is not built yet"]}'

